# ---------------------------------------------------------
# 🚨 MASTER SETUP BLOCK - DO NOT EDIT THIS CELL
# ---------------------------------------------------------

In [1]:
# ==============================================================================
# 🚀 MASTER DATA PIPELINE (LOAD -> CLEAN -> FILL)
# ==============================================================================
# Instructions:
# 1. Place 'US_Accidents_March23.csv' in a folder named 'data'.
# 2. Run the cells below in order to get a clean dataframe ready for analysis.
# ==============================================================================



In [5]:
!pip install pandas numpy matplotlib

  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.4.0-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Using cached matplotlib-3.10.8-cp312-cp312-win_amd64.whl.metadata (52 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp312-cp312-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-12.1.0-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.3.1-py3-none-any.whl.metadata (5.6 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------- -------------------------- 3.7/11.0 MB 18.2 MB/s eta 0:00:01
 

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [9]:
# ==============================================================================
# 🕵️‍♀️ DIRTY EDA (Audit the Raw Data)
# ==============================================================================
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# 1. LOAD RAW DATA
file_path = os.path.join('data', 'US_Accidents.csv')
print("⏳ Loading raw data...")
df_raw = pd.read_csv(file_path)

ImportError: C extension: pandas.util not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [ ]:
# 2. MINIMAL TYPE FIX (Required for plotting, but NO data deleted)
df_raw['Start_Time'] = pd.to_datetime(df_raw['Start_Time'], errors='coerce')
df_raw['Hour'] = df_raw['Start_Time'].dt.hour  # Create Hour just for visualization


In [ ]:
# ---------------------------------------------------------
# 3. VISUALIZE THE MESS (Missing Values)
# ---------------------------------------------------------
plt.figure(figsize=(12, 6))
sns.heatmap(df_raw.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title("Missing Value Map (Yellow = Missing)", fontsize=16)
plt.show()


 *Insight:* Look at the vertical yellow lines. Those are columns you should probably delete!

In [ ]:
# 4. CHECK FOR OUTLIERS (e.g., Temperature)
# ---------------------------------------------------------
plt.figure(figsize=(10, 4))
sns.boxplot(x=df_raw['Temperature(F)'])
plt.title("Temperature Outliers Check", fontsize=14)
plt.show()

*Insight:* If you see dots at -100 or +200, you know you need to filter them later.


In [ ]:
# 5. CHECK TARGET BALANCE (Severity)
# ---------------------------------------------------------
print("\n--- Severity Counts (Raw) ---")
print(df_raw['Severity'].value_counts())

# Cleaning the raw data

In [ ]:
# ==============================================================================
cols_to_delete = [
    # Metadata & IDs
    'ID', 'Source', 'Country', 'Zipcode', 'Timezone', 'Airport_Code', 'Description',

    # Redundant Time/Weather
    'Weather_Timestamp', 'Wind_Chill(F)', 'Wind_Direction', 'Pressure(in)', 'Precipitation(in)',
    'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight',

    # Too Granular or Mostly False
    'Street', 'Turning_Loop',

    # High Missing Values
    'End_Lat', 'End_Lng'
]
df_raw.drop(columns=cols_to_delete, errors='ignore', inplace=True)
print(f"🗑️ Dropped {len(cols_to_delete)} columns.")

In [ ]:
# ---------------------------------------------------------
# RENAME COLUMNS
# ---------------------------------------------------------
df_raw.rename(columns={
    'Distance(mi)': 'Distance',
    'Temperature(F)': 'Temp',
    'Humidity(%)': 'Humidity',
    'Visibility(mi)': 'Visibility',
    'Wind_Speed(mph)': 'Wind_Speed',
    'Weather_Condition': 'Weather'
}, inplace=True)

In [ ]:
# ---------------------------------------------------------
# FIX DATE & TIME
# ---------------------------------------------------------
print("🕒 Converting timestamps and calculating duration...")
df_raw['Start_Time'] = pd.to_datetime(df_raw['Start_Time'], errors='coerce')
df_raw['End_Time'] = pd.to_datetime(df_raw['End_Time'], errors='coerce')

# Drop rows where Start_Time or End_Time is unknown (cannot analyze without time)
df_raw.dropna(subset=['Start_Time', 'End_Time'], inplace=True)

# Create "Duration" (in Minutes)
df_raw['Duration'] = (df_raw['End_Time'] - df_raw['Start_Time']).dt.total_seconds() / 60

# Filter logic: Remove negative durations or accidents lasting > 1 week (bad data)
df_raw = df_raw[(df_raw['Duration'] > 0) & (df_raw['Duration'] < 10080)]

# Extract Temporal Features
df_raw['Year'] = df_raw['Start_Time'].dt.year
df_raw['Month'] = df_raw['Start_Time'].dt.month
df_raw['Hour'] = df_raw['Start_Time'].dt.hour
df_raw['Weekday'] = df_raw['Start_Time'].dt.day_name()

In [ ]:
# ---------------------------------------------------------
# FIX MISSING VALUES (IMPUTATION)
# ---------------------------------------------------------
print("🔧 Filling empty values...")
# A. Numerical Columns -> Fill with MEDIAN
weather_nums = ['Temp', 'Humidity', 'Visibility', 'Wind_Speed']
for col in weather_nums:
    if col in df_raw.columns:
        median_val = df_raw[col].median()
        df_raw[col] = df_raw[col].fillna(median_val)

# B. Categorical Columns -> Fill with MODE (Most Frequent)
categorical_cols = ['Weather', 'Sunrise_Sunset', 'City']
for col in categorical_cols:
    if col in df_raw.columns:
        if not df_raw[col].mode().empty:
            mode_val = df_raw[col].mode()[0]
            df_raw[col] = df_raw[col].fillna(mode_val)

In [ ]:
# ---------------------------------------------------------
# FINAL STATUS CHECK
# ---------------------------------------------------------
print("\n✅ DATA PIPELINE COMPLETE!")
print(f"Final Shape: {df_raw.shape}")
print(f"Any missing values left? {df_raw.isna().sum().sum()}")
print("------------------------------------------------")
print("Columns ready for analysis:", list(df_raw.columns))
df_raw.head(3)

In [ ]:
df_raw.to_csv("cleaned_accidents.csv", index=False)

# ==============================================================================
# 🔍 GENERAL EXPLORATORY DATA ANALYSIS (EDA)
# ==============================================================================


In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(20, 15))

In [ ]:
# ---------------------------------------------------------
# 1. TARGET VARIABLE: How bad are the accidents? (Severity)
# ---------------------------------------------------------
plt.subplot(2, 2, 1)
ax = sns.countplot(x='Severity', data=df_raw, palette='viridis')
plt.title("Distribution of Accident Severity", fontsize=14, fontweight='bold')
plt.xlabel("Severity Level (1-4)")
plt.ylabel("Count of Accidents")
# Add count labels on top of bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + 0.3, p.get_height() + 100))


In [ ]:
# ---------------------------------------------------------
# 2. TEMPORAL: When do accidents happen? (By Hour)
# ---------------------------------------------------------
plt.subplot(2, 2, 2)
sns.histplot(df_raw['Hour'], bins=24, kde=True, color='orange')
plt.title("Accidents by Hour of Day", fontsize=14, fontweight='bold')
plt.xlabel("Hour (0-23)")
plt.ylabel("Frequency")
plt.xticks(range(0, 24))


In [ ]:
# ---------------------------------------------------------
# 3. GEOSPATIAL: Which states are most dangerous? (Top 10)
# ---------------------------------------------------------
plt.subplot(2, 2, 3)
top_states = df_raw['State'].value_counts().head(10)
sns.barplot(x=top_states.values, y=top_states.index, palette='magma')
plt.title("Top 10 States by Accident Count", fontsize=14, fontweight='bold')
plt.xlabel("Number of Accidents")


In [ ]:
# ---------------------------------------------------------
# 4.CORRELATION: What numeric variables move together?
# ---------------------------------------------------------
plt.subplot(2, 2, 4)
# Select only numeric columns for correlation
numeric_cols = ['Severity', 'Distance', 'Temp', 'Humidity', 'Visibility', 'Wind_Speed', 'Duration']
corr_matrix = df_raw[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap (Numeric Variables)", fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()